# Setup

## Check Python version

In [1]:
from platform import  python_version

In [2]:
python_version()

'3.10.5'

## Install/Download necessary packages

In [3]:
import sys

In [4]:
# # Uncomment these if any packages are not installed in your current jupyter env 
# # Installing a pip package in the current kernel
# # Pandas also installs the numpy package
# !{sys.executable} -m pip install pandas  
# !{sys.executable} -m pip install requests
# !{sys.executable} -m pip install matplotlib
# !{sys.executable} -m pip install sklearn
# !{sys.executable} -m pip install featuretools

In [5]:
# import the required packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import sklearn
import os
import datetime as dt
import featuretools as ft
from featuretools.selection import selection 

# Importing and Preprocessing the Data

In [11]:
def view_files():
    path = os.getcwd()
    path = f"{path}\data"
    return(os.listdir(path))

files = view_files()
files

['20220703_120137530608_RBD.csv',
 '20220703_120253586011_RBD.xlsx',
 '20220703_120455835661_RBD.csv',
 'AllShares_growth.csv',
 'household_financial_assets-currency_and_deposits.csv',
 'investment_Qgrowth.csv',
 'inv_by_assets_intellectual.csv',
 'main.csv',
 'mainexcel.xlsx',
 'share_prices.csv',
 'test1.xlsx',
 'test1csv.csv']

## Importing and Cleaning

In [7]:
def check_file(i):
    #grab the file path from which to import the dataset
    path = os.getcwd()
    path = f"{path}\data"
    path = f"{path}\{files[i]}"
    return(path)

def import_data(path):
    # read the csv file as a dataframe and remove unnecessary columns
    df = pd.read_csv(filepath_or_buffer=path)
    return(df)
    
def clean_data(df, bank_prefix="B_34118: "):
    df = df.drop(["Unit", "Time series code"], axis="columns")
    
    # Remove ugly string labels of columns
    df.columns = df.columns.str.replace(pat="D_M_[0-9]{4}M[0-9]{2}:", repl="", regex=True)
    df["Bank"] = df["Bank"].str.replace(pat=bank_prefix, repl="", regex=True)
    df["Bank"] = df["Bank"].str.replace(pat = " ", repl = "_")
    # Remove empty title rows
    df = df[~df["Table"].str.contains("T_T[0-9]{2}R[0-9]{3}:", regex = True)]
    df = df[~df["Table"].str.contains("T_T[0-9]{2}R[0-9]{3}_A:", regex = True)]
    # Label the different tables withing the df, i.e liablities, assets, etc.
    df["Table"] = df["Table"].str.replace("T_T0[0-4]R[0-9]{3,4}C[0-9]{2}: T0[1-4]R[0-9]{3}[A]{0,1}C[0-9]{2}: ", 
                                          regex = True, repl="L_")
    df["Table"] = df["Table"].str.replace("T_T0[5]R[0-9]{3,4}C[0-9]{2}: T0[5]R[0-9]{3}[A]{0,1}C[0-9]{2}: ", 
                                      regex = True, repl="E_")
    df["Table"] = df["Table"].str.replace("T_T0[6-9]R[0-9]{3,4}C[0-9]{2}: T0[6-9]R[0-9]{3}[A]{0,1}C[0-9]{2}: ", 
                                          regex = True, repl="A_")
    df["Table"] = df["Table"].str.replace("T_T1[0-3]R[0-9]{3,4}C[0-9]{2}: T1[0-3]R[0-9]{3}[A]{0,1}C[0-9]{2}: ", 
                                          regex = True, repl="A_")
    
    df["Table"] = df["Table"].str.replace("[(][0-9a-z\s,]{2,}[)][:] ", regex = True, repl="")
    #
    df = df.set_index("Table").T.drop("Bank")
    df.index = pd.to_datetime(df.index, format=" %YM%m")
    df.dropna(axis=1, inplace=True)
    df.columns = df.columns.str.replace(pat=" ", repl="_")
    df = df.apply(pd.to_numeric)
    df = ft.selection.remove_highly_null_features(df)
    df = ft.selection.remove_single_value_features(df)
    df = df.loc[:,~df.columns.duplicated()]    
    
    return(df)

In [8]:
check_file(3)

'C:\\GitHub\\DS_PROJ\\data\\AllShares_growth.csv'

In [9]:
import_data(check_file(3)).head(3)

,DATE,SPASTT01ZAM657N
0,1960-02-01,-1.459491
1,1960-03-01,-7.054686
2,1960-04-01,-9.074221


The data is stored in an ugly format, and needs too be transformed and formatted into
a more usable form for data analysis
Below the data is:
* Transformed into the standard dataframe format
* Column names are reformatted
* Observation types are converted to floats
* The date variable is transformed to a datetime object for ease of use
* The correct labels for liabilities, assets, and equity is assigned

In [10]:
absa = clean_data(import_data(check_file(3)))

KeyError: "['Unit', 'Time series code'] not found in axis"

In [ ]:
absa.head(1)

Some of the highly overlapping columns are removed now, 
and others will be removed after the necessary
additional features have been calculated.

In [128]:
# This dataset has a particular problem with overlapping or highly
# correlated features that contain elements from other columns
# Thus, we remove these highly correlated features

def remove_corr(df):
    df.reset_index(inplace=True)
    es = ft.EntitySet(id="Absa_BS")
    ent_set = es.add_dataframe(dataframe_name="Absa_Group_Ltd",
                                dataframe=df, 
                                already_sorted=False, index = "index")
    
    fm, features = ft.dfs(entityset=ent_set,
                      target_dataframe_name="Absa_Group_Ltd",
                      trans_primitives=[],
                      agg_primitives=[], 
                      max_depth=1)
    # From experimentation, the 0.97 threshold seems to remove the 
    # columns that are verbatim totals of others and not removing
    # other columns required for feature analysis.
    # A lower threshold will be used at a later stage after some
    # columns have been used in calculation of additional 
    # features
    fm = ft.selection.remove_highly_correlated_features(fm, 
                                                        pct_corr_threshold=0.97)
    fm.reset_index(inplace=True)
    fm.set_index("index", inplace=True)
    fm.columns.name = "Date"
    fm.index.name = None

    return(fm)

absa = remove_corr(absa)

### Creating the Liquidity Ratio Variable

Now, we want to create a loan to deposit ratio to create a liquidity ratio variable for the bank

In [129]:
absa.iloc[0,absa.columns.str.contains("^L.*deposits", case=False)]#[1:8]#.sum()

Date
L_DEPOSITS_Cheque_(1)                                                                      9588342.0
L_DEPOSITS_Savings_(2)                                                                     7800740.0
L_DEPOSITS_Up_to_1_day_(3)                                                                 8946659.0
L_DEPOSITS_More_than_1_day_to_1_month_(4)                                                  9417927.0
L_DEPOSITS_More_than_1_month_to_6_months_(5)                                              21301640.0
L_DEPOSITS_More_than_6_months_(6)                                                          8981624.0
L_Other_deposits:_More_than_1_day_to_1_month_(4)                                            342393.0
L_Other_deposits:_More_than_1_month_to_6_months_(5)                                         435657.0
L_Other_deposits:_More_than_6_months_(6)                                                    171516.0
L_Other_deposits:_TOTAL_(7)                                                           

In [130]:
absa.iloc[0,absa.columns.str.contains("LOANS", case=False, regex=True)]

Date
A_Other_deposits_with_and_loans_and_advances_to_SA_banksb:_Domestic_assets_(1)                       501582.0
A_Foreign_currency_loans_and_advances_TOTAL_ASSETS_(Col_1_plus_col_3)_(5)                            823789.0
A_Overdrafts,_loans_and_advances:_public_sector_Domestic_assets_(1)                                  575480.0
A_Overdrafts,_loans_and_advances:_public_sector_TOTAL_ASSETS_(Col_1_plus_col_3)_(5)                  575578.0
A_Less:_credit_impairments_in_respect_of_loans_and_advances:_TOTAL_ASSETS_(Col_1_plus_col_3)_(5)    1995422.0
Name: 1993-01-01 00:00:00, dtype: float64

In [131]:
def create_total_deposits(df_global):
    df = df_global
    deposits = df.columns.str.contains("^L.*deposits", 
                                           case=False, 
                                           regex=True)
    drops = list(df.columns[deposits])
    df["L_TOTAL_DEPOSITS"] = df.iloc[:,deposits].sum(axis=1)
    df = df.drop(columns=drops)
    return(df)



In [132]:
absa = create_total_deposits(absa)

In [133]:
def create_total_loans(df_global):
    df = df_global
    loans = df.columns.str.contains("Loans", 
                                       case=False, 
                                       regex=True)
    drops = list(df.columns[loans])
    df["A_TOTAL_LOANS"] = df.iloc[:,loans].sum(axis=1)
    df = df.drop(columns=drops)
    return(df)


In [134]:
absa = create_total_loans(absa)

In [138]:
def liquidity_ratio(df):
    
    

absa[["A_TOTAL_LOANS", "L_TOTAL_DEPOSITS"]]

Date,A_TOTAL_LOANS,L_TOTAL_DEPOSITS
1993-01-01,4471851.0,7.210242e+07
1993-02-01,7615276.0,7.023894e+07
1993-03-01,6410475.0,7.228946e+07
1993-04-01,4496268.0,6.939787e+07
1993-05-01,4452967.0,6.757947e+07
...,...,...
2021-12-01,47481729.0,1.199028e+09
2022-01-01,42900565.0,1.153581e+09
2022-02-01,53452861.0,1.172717e+09
2022-03-01,45123231.0,1.185268e+09


In [ ]:
remove2 = list()
for h in range(len(absa.columns.to_list())):
    for i in range(1, round(0.2*len(absa.columns.to_list()))):
    if all(round(absa.iloc[:,h], 
                 ndigits=-2) == round(absa.iloc[:,h+1:i].sum(axis=1), 
                                      ndigits=-2)):
        remove2.extend(absa.iloc[:,h+1:i].columns.to_list())
        print(absa.iloc[:,h+1:i].columns.to_list())
    else:
        continue

len(set(remove2)), len(remove2) # The list is unique

total_mod_BA900.drop(remove2, axis=1, inplace=True)

# remove3 = list()

# for h in range(len(total_mod_BA900.columns.to_list())):
# for i in range(1,
# round(0.5*len(total_mod_BA900.columns.to_list()))):
# if all(round(total_mod_BA900.iloc[:,h], ndigits=-2) ==
# round(total_mod_BA900.iloc[:,h+1:i].sum(axis=1), ndigits=-2)):
# remove3.extend(total_mod_BA900.iloc[:,h+1:i].columns.to_list())
# print(total_mod_BA900.iloc[:,h+1:i].columns.to_list())
# else:
# continuetotal_mod_BA900.drop(remove3, axis=1, inplace=True)remove4 = list()
# for h in
# range(len(total_mod_BA900.columns.to_list())):
# for i in range(1, round(0.5*len(total_mod_BA900.columns.to_list()))):
# if
# all(round(total_mod_BA900.iloc[:,h], ndigits=-2) == round(total_mod_BA900.iloc[:,h+1:i].sum(axis=1), ndigits=-2)):
# remove4.extend(total_mod_BA900.iloc[:,h+1:i].columns.to_list())
# print(total_mod_BA900.iloc[:,h+1:i].columns.to_list())
# else:
# continuetotal_mod_BA900.drop(remove4, axis=1, inplace=True) #.filter(regex="L_OtherDomestic[AA-Zaz_]*_
# Total")["L_OtherDomestic_Total"]remove5 = list()
# for h in range(len(total_mod_BA900.columns.to_list())):
# for i in
# range(1, round(0.25*len(total_mod_BA900.columns.to_list()))):
# if all(round(total_mod_BA900.iloc[:,h], ndigits=-2) ==
# round(total_mod_BA900.iloc[:,h+1:i].sum(axis=1), ndigits=-2)):
# remove5.extend(total_mod_BA900.iloc[:,h+1:i].columns.to_list())
# print(total_mod_BA900.iloc[:,h+1:i].columns.to_list())

some of these columns seem to appear 'twice' and will therefore be removed

## Removing Highly Correlated Features

This dataset contains groups of variables that sum together to form larger aggregates. To ensure
that the model only uses features that are relevant once, we remove a subset of those features that
are too highly correlated with others. This would also introduce the issue of multicolinearity if the
features are not removed.

In [ ]:
# This dataset has a particular problem with overlapping or highly
# correlated features that contain elements from other columns
# Thus, we remove these highly correlated features

def remove_corr(df):
    df.reset_index(inplace=True)
    es = ft.EntitySet(id="Absa_BS")
    ent_set = es.add_dataframe(dataframe_name="Absa_Group_Ltd",
                                dataframe=df, 
                                already_sorted=False, index = "index")
    
    fm, features = ft.dfs(entityset=ent_set,
                      target_dataframe_name="Absa_Group_Ltd",
                      trans_primitives=[],
                      agg_primitives=[], 
                      max_depth=1)
    # From experimentation, the 0.9825 threshold seems to remove the 
    # columns that are verbatim totals of others
    # A lower threshold will be used at a later stage after some
    # columns have been used in calculation of additional 
    # features
    fm = ft.selection.remove_highly_correlated_features(fm, 
                                                        pct_corr_threshold=0.9825)
    fm.reset_index(inplace=True)
    fm.set_index("index", inplace=True)
    fm.columns.name = "Date"
    fm.index.name = None

    return(fm)

absa = remove_corr(absa)

In [ ]:
absa.info()